In [38]:
import numpy as np

**Dieferenças Finitas na Equação de Poisson**

In [39]:
a, b = 0, 2  # Intervalo em x
c, d = 0, 1  # Intervalo em y

def f(x, y):
    return x * np.exp(y)

def g(x, y):
    if x == a: 
        return 0
    elif x == b:
        return 2 * np.exp(y)
    elif y == c:
        return x
    elif y == d:
        return np.e*x
    return 0  

def finite_difference(f, g, n, m, e = 1e-10, max_i = 1000):
    # Passo 1
    h = (b - a) / n
    k = (d - c) / m

    # Passo 2 e 3
    x = np.zeros(n)
    y = np.zeros(m)

    for i in range(n): x[i] = a + i*h
    for i in range(m): y[i] = c + i*k

    # Passo 4
    W = np.zeros((n+1, m+1))

    # Passo 5
    lamb = (h**2) / (k**2)
    u = 2 * (1 + lamb)

    # Passo 6
    for l in range(1, max_i + 1):

        # Passo 7
        z = (-(h**2) * f(x[1],y[m-1]) + g(a,y[m-1]) + lamb*g(x[1],d) + lamb * W[1,m-2] +  W[2,m-1]) / u
        norm = abs(z - W[1,m-1])    
        W[1,m-1] = z

        # Passo 8
        for i in range(2, n-1):
            z = (-(h**2) * f(x[i],y[m-1]) + lamb*g(x[i],d) + W[i-1,m-1] + W[i+1,m-1] + lamb*W[i,m-2]) / u
            if abs(z - W[i,m-1]) > norm:
                norm = abs(z - W[i,m-1])
            W[i,m-1] = z

        # Passo 9
        z = (-(h**2) * f(x[n-1],y[m-1]) + g(b,y[m-1]) + lamb*g(x[n-1],d) + W[n-2,m-1] +  lamb*W[n-1,m-2]) / u
        if abs(W[n-1,m-1] - z) > norm:
            norm = abs(W[n-1,m-1] - z)
        W[n-1,m-1] = z

        # Passo 10
        for j in range(m-2, 1,-1):

            # Passo 11
            z = (-(h**2) * f(x[1],y[j]) + g(a,y[j]) + lamb * W[1,j+1] + lamb * W[1,j-1] + W[2,j]) / u
            if abs(W[1,j] - z) > norm:
                norm = abs(W[1,j] - z)
            W[1,j] = z

            # Passo 12
            for i in range(2, n-1):
                z = (-(h**2) * f(x[i],y[j]) + W[i-1,j] + lamb * W[i,j+1] + W[i+1,j] + lamb * W[i,j-1]) / u
                if abs(W[i,j] - z) > norm:
                    norm = abs(W[i,j] - z)
                W[i,j] = z

            # Passo 13
            z = (-(h**2) * f(x[n-1],y[j]) + g(b,y[j]) + W[n-2,j] + lamb * W[n-1,j+1] + lamb * W[n-1,j-1]) / u
            if abs(W[n-1,j] - z) > norm:
                norm = abs(W[n-1,j] - z)
            W[n-1,j] = z

        # Passo 14
        z = (-(h**2) * f(x[1],y[1]) + g(a,y[1]) + lamb * g(x[1],c) + lamb * W[1,2] + W[2,1]) / u
        if abs(W[1,1] - z) > norm:
            norm = abs(W[1,1] - z)
        W[1,1] = z

        # Passo 15
        for i in range(2, n-1):
            z = (-(h**2) * f(x[i],y[1]) + lamb*g(x[i],c) + W[i-1,1] +  W[i+1,1] + lamb*W[i,2]) / u
            if abs(W[i,1] - z) > norm:
                norm = abs(z - W[i,1])
            W[i,1] = z
        
        # Passo 16
        z = (-(h**2) * f(x[n-1],y[1]) + g(b,y[1]) + lamb * g(x[n-1],c) + W[n-2,1] + lamb * W[n-1,2]) / u
        if abs(W[n-1,1] - z) > norm:
            norm = abs(W[n-1,1] - z)
        W[n-1,1] = z

        # Passo 17
        if norm <= e: break

        l+=1
    return W
    
n, m = 6, 5

w = finite_difference(f, g, n, m)
print(w)


[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.40726462 0.49748325 0.60759608 0.74200707 0.        ]
 [0.         0.81452375 0.99495758 1.21518318 1.48400854 0.        ]
 [0.         1.22176621 1.49240473 1.82274271 2.2259927  0.        ]
 [0.         1.62896369 1.9897783  2.43022656 2.96792825 0.        ]
 [0.         2.03604232 2.48695839 3.0375055  3.70972407 0.        ]
 [0.         0.         0.         0.         0.         0.        ]]
